In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn import metrics
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [2]:
train=pd.read_csv("train_LZdllcl.csv")
test=pd.read_csv("test_2umaH9m.csv")
sub=pd.read_csv("sample_submission_M0L0uXE.csv")

In [3]:
train["education"]=train["education"].fillna("Bachelor's")
train["previous_year_rating"]=train["previous_year_rating"].fillna(3)

test["education"]=test["education"].fillna("Bachelor's")
test["previous_year_rating"]=test["previous_year_rating"].fillna(3)


In [4]:
s= StandardScaler()
X_std = s.fit_transform(train[["no_of_trainings","age","previous_year_rating","length_of_service","avg_training_score"]])

X2=pd.get_dummies(train[["department","region","education","gender","recruitment_channel"]])
y = train["is_promoted"]
X2=pd.concat([train[["KPIs_met >80%","awards_won?"]],X2],axis=1)
X=np.concatenate([X_std,X2],axis=1)

In [5]:
train_X,val_X,train_y,val_y=train_test_split(X,y,test_size=0.3,random_state=42)

In [6]:
sm=SMOTE(random_state=42)
train_X_sm,train_y_sm = sm.fit_sample(train_X,train_y)

In [7]:
xgb=XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=500, silent=True, objective='binary:logistic', 
                  booster='gbtree', n_jobs=-1, nthread=4, gamma=0, min_child_weight=1, max_delta_step=0, 
                  subsample=0.9, colsample_bytree=0.9, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, 
                  scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None)

In [8]:
model_xgb=xgb.fit(train_X_sm,train_y_sm)
predictions_xgb=model_xgb.predict(val_X)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
metrics.f1_score(val_y,predictions_xgb)

0.5058823529411764

In [10]:
test_std = s.transform(test[["no_of_trainings","age","previous_year_rating","length_of_service","avg_training_score"]])
test_X=pd.get_dummies(test[["department","region","education","gender","recruitment_channel"]])
test_X=pd.concat([test[["KPIs_met >80%","awards_won?"]],test_X],axis=1)
test_X=np.concatenate([test_std,test_X.values],axis=1)

In [11]:
predictions_test=model_xgb.predict(test_X)
sub4=pd.DataFrame({"employee_id":test.employee_id,"is_promoted":predictions_test})
sub4.to_csv("sub4.csv",index=False)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
